# Idea used.

The main aim here is to train the model using noisy labels. Role of label is as important in training of data as is the data itself. Many a times we observe incorrect labels can lead to cause improper training. If I were to use an analogy suppose if a student is taught from nursery that 'A'---This alphabet is K and not an A. Then again if someone again tells him 'K' is K and 'A' is A. He is sure to be confused. Same is here. If we do not cleanse our data and properly feed the label,our model will surely fail. In this model I used the most basic technique used in noisy label training and am exploring more to widen my ideas. So here we take 2 models. This was actually a very basic idea used by Mr. Reed in begining of his idea of channel wise training with Adaptation layer. So the idea is take a few data which you are sure is absolutely correct, train a small network based on this dataset and use it to actually predict the labels of noisy model.
P(noisy datasets/basic_model is true)=P(noisy_datset ^ basic model)/(P basic model).....^ is intersection here. So now if we use the same model to find out correct data then the probability will be not very accurate. So it is better to train on a set which is sure to give high accuracy. Here I have used the MNIST datset for the following. So let us start with importing modules necessary

# Importing all the necessary files and modules.

In [60]:
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D,Conv2D, MaxPooling2D
from keras.utils import np_utils,to_categorical
from keras.datasets import mnist
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

# Load the MNIST training and testing data

In [44]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_size=1*28*28;
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
X_train = X_train.reshape(X_train.shape[0], img_size)
X_test = X_test.reshape(X_test.shape[0], img_size)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.
print(X_train.shape,y_train.shape)

(60000, 784) (60000, 10)


# Creating S_model

S_model stands for simple model. I have used an OOP approach so that we can reuse it later by importing this class in other files or inheriting it. So here I have taken the mnist test and train set along with their output values. Passed them through a network of 2 hidden layers consisting 300 and 100 perceptrons. Also since mnist dataset is stored in batches hence, we had to convert the outputs into its respective categories so that we can specify last layer(output layer) having 10 values and train each perceptron accordingly. Next is returning the model for further use.

In [28]:
class s_model:
    def __init__(self):
        self.ganguly=Sequential();
    def one_hot(self,value,size):
        arr=np.array([0]*size);
        arr[value]=1;
        return arr;
    def train(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        y_train=to_categorical(y_train)
        y_test=to_categorical(y_test)
        img_size=1*28*28;
        X_train = X_train.reshape(X_train.shape[0], img_size)
        X_test = X_test.reshape(X_test.shape[0], img_size)
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255.
        X_test /= 255.
        X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.1)
        X_train_arr=np.array(X_train)
        X_val_arr=np.array(X_val)
        y_train_arr=np.array(y_train)
        y_val_arr=np.array(y_val)
        w,d=X_train_arr.shape
        self.ganguly.add(Dense(350,activation='relu', input_shape=(d,)))
        self.ganguly.add(Dropout(0.2))
        self.ganguly.add(Dense(100,activation='relu', input_shape=(d,)))
        self.ganguly.add(Dropout(0.5))
        self.ganguly.add(Dense(10, activation='softmax'))
        self.ganguly.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        feeds=self.ganguly.fit(x=X_train_arr,y=y_train_arr,epochs=30,validation_data=(X_val_arr,y_val_arr),batch_size=256,verbose=True)
        print("Training model is complete")
    def get_model(self):
        self.train();
        return self.ganguly;

# Creation of object for above class

In [29]:
obj=s_model();
model=obj.get_model();

Train on 54000 samples, validate on 6000 samples
Epoch 1/30
54000/54000 [==============================] - ETA: 59:11 - loss: 2.4529 - acc: 0.08 - ETA: 29:32 - loss: 2.3959 - acc: 0.10 - ETA: 5:42 - loss: 2.0139 - acc: 0.3273 - ETA: 3:14 - loss: 1.7187 - acc: 0.448 - ETA: 2:34 - loss: 1.5859 - acc: 0.495 - ETA: 2:02 - loss: 1.4562 - acc: 0.535 - ETA: 1:48 - loss: 1.3867 - acc: 0.558 - ETA: 1:33 - loss: 1.3004 - acc: 0.588 - ETA: 1:21 - loss: 1.2350 - acc: 0.610 - ETA: 1:12 - loss: 1.1722 - acc: 0.630 - ETA: 1:04 - loss: 1.1245 - acc: 0.645 - ETA: 57s - loss: 1.0813 - acc: 0.660 - ETA: 53s - loss: 1.0523 - acc: 0.66 - ETA: 48s - loss: 1.0169 - acc: 0.68 - ETA: 44s - loss: 0.9854 - acc: 0.69 - ETA: 39s - loss: 0.9467 - acc: 0.70 - ETA: 35s - loss: 0.9127 - acc: 0.71 - ETA: 32s - loss: 0.8809 - acc: 0.72 - ETA: 29s - loss: 0.8497 - acc: 0.73 - ETA: 26s - loss: 0.8276 - acc: 0.74 - ETA: 24s - loss: 0.8029 - acc: 0.75 - ETA: 22s - loss: 0.7819 - acc: 0.75 - ETA: 20s - loss: 0.7625 - acc: 0.

54000/54000 [==============================] - ETA: 4s - loss: 0.1190 - acc: 0.964 - ETA: 3s - loss: 0.1184 - acc: 0.963 - ETA: 3s - loss: 0.1208 - acc: 0.966 - ETA: 3s - loss: 0.1144 - acc: 0.966 - ETA: 3s - loss: 0.1195 - acc: 0.965 - ETA: 3s - loss: 0.1184 - acc: 0.967 - ETA: 3s - loss: 0.1171 - acc: 0.967 - ETA: 3s - loss: 0.1206 - acc: 0.965 - ETA: 3s - loss: 0.1215 - acc: 0.965 - ETA: 3s - loss: 0.1252 - acc: 0.964 - ETA: 3s - loss: 0.1228 - acc: 0.965 - ETA: 3s - loss: 0.1212 - acc: 0.965 - ETA: 3s - loss: 0.1194 - acc: 0.966 - ETA: 3s - loss: 0.1191 - acc: 0.965 - ETA: 3s - loss: 0.1199 - acc: 0.965 - ETA: 3s - loss: 0.1187 - acc: 0.965 - ETA: 2s - loss: 0.1174 - acc: 0.966 - ETA: 2s - loss: 0.1182 - acc: 0.966 - ETA: 2s - loss: 0.1184 - acc: 0.965 - ETA: 2s - loss: 0.1164 - acc: 0.966 - ETA: 2s - loss: 0.1188 - acc: 0.965 - ETA: 2s - loss: 0.1201 - acc: 0.965 - ETA: 2s - loss: 0.1207 - acc: 0.965 - ETA: 2s - loss: 0.1193 - acc: 0.965 - ETA: 2s - loss: 0.1196 - acc: 0.965 - ETA

54000/54000 [==============================] - ETA: 4s - loss: 0.0732 - acc: 0.984 - ETA: 3s - loss: 0.0796 - acc: 0.973 - ETA: 3s - loss: 0.0701 - acc: 0.977 - ETA: 3s - loss: 0.0745 - acc: 0.977 - ETA: 3s - loss: 0.0741 - acc: 0.978 - ETA: 3s - loss: 0.0747 - acc: 0.977 - ETA: 3s - loss: 0.0722 - acc: 0.979 - ETA: 3s - loss: 0.0704 - acc: 0.980 - ETA: 3s - loss: 0.0695 - acc: 0.980 - ETA: 3s - loss: 0.0692 - acc: 0.980 - ETA: 3s - loss: 0.0680 - acc: 0.980 - ETA: 3s - loss: 0.0681 - acc: 0.980 - ETA: 3s - loss: 0.0675 - acc: 0.980 - ETA: 3s - loss: 0.0684 - acc: 0.980 - ETA: 3s - loss: 0.0690 - acc: 0.980 - ETA: 3s - loss: 0.0688 - acc: 0.980 - ETA: 3s - loss: 0.0672 - acc: 0.980 - ETA: 2s - loss: 0.0671 - acc: 0.980 - ETA: 2s - loss: 0.0670 - acc: 0.980 - ETA: 2s - loss: 0.0675 - acc: 0.980 - ETA: 2s - loss: 0.0671 - acc: 0.980 - ETA: 2s - loss: 0.0687 - acc: 0.980 - ETA: 2s - loss: 0.0683 - acc: 0.979 - ETA: 2s - loss: 0.0696 - acc: 0.979 - ETA: 2s - loss: 0.0694 - acc: 0.979 - ETA

54000/54000 [==============================] - ETA: 5s - loss: 0.0657 - acc: 0.980 - ETA: 4s - loss: 0.0478 - acc: 0.988 - ETA: 4s - loss: 0.0473 - acc: 0.986 - ETA: 4s - loss: 0.0474 - acc: 0.984 - ETA: 4s - loss: 0.0461 - acc: 0.985 - ETA: 4s - loss: 0.0453 - acc: 0.987 - ETA: 3s - loss: 0.0474 - acc: 0.986 - ETA: 3s - loss: 0.0455 - acc: 0.987 - ETA: 3s - loss: 0.0451 - acc: 0.987 - ETA: 3s - loss: 0.0462 - acc: 0.987 - ETA: 3s - loss: 0.0456 - acc: 0.987 - ETA: 3s - loss: 0.0453 - acc: 0.987 - ETA: 3s - loss: 0.0467 - acc: 0.986 - ETA: 3s - loss: 0.0466 - acc: 0.986 - ETA: 3s - loss: 0.0469 - acc: 0.986 - ETA: 3s - loss: 0.0477 - acc: 0.986 - ETA: 3s - loss: 0.0474 - acc: 0.986 - ETA: 3s - loss: 0.0471 - acc: 0.986 - ETA: 3s - loss: 0.0465 - acc: 0.986 - ETA: 3s - loss: 0.0463 - acc: 0.987 - ETA: 3s - loss: 0.0476 - acc: 0.986 - ETA: 3s - loss: 0.0469 - acc: 0.986 - ETA: 3s - loss: 0.0470 - acc: 0.986 - ETA: 2s - loss: 0.0473 - acc: 0.986 - ETA: 2s - loss: 0.0473 - acc: 0.986 - ETA

54000/54000 [==============================] - ETA: 4s - loss: 0.0295 - acc: 0.988 - ETA: 3s - loss: 0.0327 - acc: 0.989 - ETA: 3s - loss: 0.0323 - acc: 0.990 - ETA: 3s - loss: 0.0381 - acc: 0.987 - ETA: 3s - loss: 0.0390 - acc: 0.987 - ETA: 3s - loss: 0.0385 - acc: 0.987 - ETA: 3s - loss: 0.0397 - acc: 0.986 - ETA: 3s - loss: 0.0404 - acc: 0.986 - ETA: 3s - loss: 0.0406 - acc: 0.986 - ETA: 3s - loss: 0.0403 - acc: 0.986 - ETA: 3s - loss: 0.0404 - acc: 0.986 - ETA: 3s - loss: 0.0402 - acc: 0.986 - ETA: 3s - loss: 0.0394 - acc: 0.987 - ETA: 3s - loss: 0.0394 - acc: 0.986 - ETA: 2s - loss: 0.0392 - acc: 0.986 - ETA: 2s - loss: 0.0388 - acc: 0.987 - ETA: 2s - loss: 0.0391 - acc: 0.987 - ETA: 2s - loss: 0.0392 - acc: 0.987 - ETA: 2s - loss: 0.0392 - acc: 0.987 - ETA: 2s - loss: 0.0386 - acc: 0.987 - ETA: 2s - loss: 0.0390 - acc: 0.987 - ETA: 2s - loss: 0.0384 - acc: 0.987 - ETA: 2s - loss: 0.0385 - acc: 0.987 - ETA: 2s - loss: 0.0386 - acc: 0.987 - ETA: 2s - loss: 0.0387 - acc: 0.987 - ETA

...

54000/54000 [==============================] - ETA: 3s - loss: 0.0228 - acc: 0.996 - ETA: 2s - loss: 0.0230 - acc: 0.993 - ETA: 2s - loss: 0.0212 - acc: 0.994 - ETA: 2s - loss: 0.0249 - acc: 0.993 - ETA: 2s - loss: 0.0264 - acc: 0.992 - ETA: 2s - loss: 0.0250 - acc: 0.992 - ETA: 2s - loss: 0.0245 - acc: 0.992 - ETA: 2s - loss: 0.0255 - acc: 0.992 - ETA: 1s - loss: 0.0242 - acc: 0.992 - ETA: 1s - loss: 0.0239 - acc: 0.993 - ETA: 1s - loss: 0.0253 - acc: 0.992 - ETA: 1s - loss: 0.0259 - acc: 0.992 - ETA: 1s - loss: 0.0263 - acc: 0.992 - ETA: 1s - loss: 0.0256 - acc: 0.992 - ETA: 1s - loss: 0.0258 - acc: 0.992 - ETA: 1s - loss: 0.0258 - acc: 0.992 - ETA: 1s - loss: 0.0266 - acc: 0.992 - ETA: 1s - loss: 0.0267 - acc: 0.992 - ETA: 1s - loss: 0.0270 - acc: 0.992 - ETA: 1s - loss: 0.0273 - acc: 0.992 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0276 - acc: 0.991 - ETA: 1s - loss: 0.0272 - acc: 0.991 - ETA: 1s - loss: 0.0269 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA

54000/54000 [==============================] - ETA: 3s - loss: 0.0039 - acc: 1.000 - ETA: 3s - loss: 0.0194 - acc: 0.993 - ETA: 2s - loss: 0.0209 - acc: 0.992 - ETA: 2s - loss: 0.0226 - acc: 0.992 - ETA: 2s - loss: 0.0211 - acc: 0.992 - ETA: 2s - loss: 0.0206 - acc: 0.992 - ETA: 2s - loss: 0.0195 - acc: 0.992 - ETA: 2s - loss: 0.0190 - acc: 0.993 - ETA: 2s - loss: 0.0196 - acc: 0.992 - ETA: 2s - loss: 0.0208 - acc: 0.992 - ETA: 2s - loss: 0.0211 - acc: 0.992 - ETA: 2s - loss: 0.0210 - acc: 0.992 - ETA: 2s - loss: 0.0215 - acc: 0.992 - ETA: 2s - loss: 0.0212 - acc: 0.992 - ETA: 2s - loss: 0.0206 - acc: 0.992 - ETA: 2s - loss: 0.0207 - acc: 0.992 - ETA: 2s - loss: 0.0206 - acc: 0.992 - ETA: 2s - loss: 0.0203 - acc: 0.993 - ETA: 2s - loss: 0.0202 - acc: 0.993 - ETA: 2s - loss: 0.0200 - acc: 0.993 - ETA: 1s - loss: 0.0207 - acc: 0.993 - ETA: 1s - loss: 0.0204 - acc: 0.993 - ETA: 1s - loss: 0.0208 - acc: 0.993 - ETA: 1s - loss: 0.0208 - acc: 0.993 - ETA: 1s - loss: 0.0211 - acc: 0.993 - ETA

54000/54000 [==============================] - ETA: 3s - loss: 0.0078 - acc: 1.000 - ETA: 3s - loss: 0.0165 - acc: 0.997 - ETA: 3s - loss: 0.0135 - acc: 0.997 - ETA: 3s - loss: 0.0170 - acc: 0.996 - ETA: 3s - loss: 0.0156 - acc: 0.996 - ETA: 3s - loss: 0.0162 - acc: 0.995 - ETA: 3s - loss: 0.0173 - acc: 0.995 - ETA: 3s - loss: 0.0172 - acc: 0.995 - ETA: 3s - loss: 0.0161 - acc: 0.995 - ETA: 3s - loss: 0.0152 - acc: 0.995 - ETA: 3s - loss: 0.0174 - acc: 0.995 - ETA: 3s - loss: 0.0182 - acc: 0.994 - ETA: 3s - loss: 0.0184 - acc: 0.994 - ETA: 3s - loss: 0.0183 - acc: 0.994 - ETA: 3s - loss: 0.0178 - acc: 0.994 - ETA: 3s - loss: 0.0185 - acc: 0.994 - ETA: 3s - loss: 0.0182 - acc: 0.994 - ETA: 2s - loss: 0.0186 - acc: 0.994 - ETA: 2s - loss: 0.0192 - acc: 0.993 - ETA: 2s - loss: 0.0191 - acc: 0.993 - ETA: 2s - loss: 0.0190 - acc: 0.993 - ETA: 2s - loss: 0.0192 - acc: 0.993 - ETA: 2s - loss: 0.0199 - acc: 0.993 - ETA: 2s - loss: 0.0199 - acc: 0.993 - ETA: 2s - loss: 0.0200 - acc: 0.993 - ETA

54000/54000 [==============================] - ETA: 4s - loss: 0.0091 - acc: 0.996 - ETA: 3s - loss: 0.0205 - acc: 0.993 - ETA: 3s - loss: 0.0168 - acc: 0.995 - ETA: 3s - loss: 0.0149 - acc: 0.996 - ETA: 3s - loss: 0.0172 - acc: 0.996 - ETA: 3s - loss: 0.0172 - acc: 0.995 - ETA: 3s - loss: 0.0200 - acc: 0.994 - ETA: 3s - loss: 0.0210 - acc: 0.994 - ETA: 3s - loss: 0.0193 - acc: 0.995 - ETA: 3s - loss: 0.0195 - acc: 0.995 - ETA: 3s - loss: 0.0196 - acc: 0.995 - ETA: 3s - loss: 0.0196 - acc: 0.994 - ETA: 3s - loss: 0.0196 - acc: 0.994 - ETA: 3s - loss: 0.0200 - acc: 0.994 - ETA: 3s - loss: 0.0196 - acc: 0.994 - ETA: 3s - loss: 0.0192 - acc: 0.994 - ETA: 2s - loss: 0.0188 - acc: 0.994 - ETA: 2s - loss: 0.0190 - acc: 0.994 - ETA: 2s - loss: 0.0194 - acc: 0.994 - ETA: 2s - loss: 0.0192 - acc: 0.994 - ETA: 2s - loss: 0.0196 - acc: 0.994 - ETA: 2s - loss: 0.0192 - acc: 0.994 - ETA: 2s - loss: 0.0189 - acc: 0.994 - ETA: 2s - loss: 0.0186 - acc: 0.994 - ETA: 2s - loss: 0.0187 - acc: 0.994 - ETA

54000/54000 [==============================] - ETA: 4s - loss: 0.0107 - acc: 1.000 - ETA: 3s - loss: 0.0123 - acc: 0.996 - ETA: 3s - loss: 0.0150 - acc: 0.995 - ETA: 3s - loss: 0.0159 - acc: 0.995 - ETA: 3s - loss: 0.0174 - acc: 0.994 - ETA: 3s - loss: 0.0157 - acc: 0.994 - ETA: 3s - loss: 0.0151 - acc: 0.995 - ETA: 3s - loss: 0.0161 - acc: 0.995 - ETA: 3s - loss: 0.0177 - acc: 0.994 - ETA: 3s - loss: 0.0206 - acc: 0.993 - ETA: 3s - loss: 0.0200 - acc: 0.994 - ETA: 3s - loss: 0.0202 - acc: 0.994 - ETA: 3s - loss: 0.0195 - acc: 0.994 - ETA: 3s - loss: 0.0199 - acc: 0.993 - ETA: 3s - loss: 0.0203 - acc: 0.993 - ETA: 3s - loss: 0.0204 - acc: 0.993 - ETA: 3s - loss: 0.0204 - acc: 0.993 - ETA: 3s - loss: 0.0199 - acc: 0.994 - ETA: 2s - loss: 0.0199 - acc: 0.993 - ETA: 2s - loss: 0.0195 - acc: 0.994 - ETA: 2s - loss: 0.0193 - acc: 0.994 - ETA: 2s - loss: 0.0197 - acc: 0.993 - ETA: 2s - loss: 0.0201 - acc: 0.993 - ETA: 2s - loss: 0.0198 - acc: 0.993 - ETA: 2s - loss: 0.0198 - acc: 0.993 - ETA

# Getting corrected labels

Ok now that we have done training our simple network, we need to import the given data and correct its labels with the previous network. So for that we create a function named as get_correct_label(). It passes each image through the network to get the correct label and store it in y list

In [32]:
def get_correct_label(model,image,object_created):
    img1=np.reshape((cv2.resize(image,(28,28))),(28*28*1))
    predict=model.predict(np.array([img1]))[0]
    acc=max(predict)
    val=np.argmax(predict)
    return object_created.one_hot(val,10);

In [34]:
X,y=list(),list();#define empty list for loading images and output one-hot vector
path="D://datasets//IIIT_D//mnistTask";#path where the dataset is downloaded in my machine
fol=os.listdir(path)#list of files in the 'train' folder in the location D:\datasets\IIIT_D\mnistTask
size=10;
flag=0;
for val,i in tqdm(enumerate(fol)):
    path_n=os.path.join(path,i);
    for img_n in os.listdir(path_n):
        img=cv2.imread(os.path.join(path_n,img_n),cv2.IMREAD_GRAYSCALE)/255#normalize by dividing by 255 for better results.
        X.append((np.reshape(cv2.resize(img,(28,28)),(28*28*1))))#initially we obtained a 2d array. In order to fit it in our keras CNN model, we had to convert it into a 3d model. Also the image is downsized to avoid OOM (Out of Memory) exception
        y.append(get_correct_label(model,img,obj))#store our results by converting it into a one-hot vector. Here the enumerate () returning counter val palys a very important role.

10it [08:20, 60.28s/it]


# Splitting up of training data

For better training we can use cross_validation set to improve the accuracy by creating cross validation network

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.15, random_state=42)
X_train_arr=np.array(X_train)
X_val_arr=np.array(X_val)
y_train_arr=np.array(y_train)
y_val_arr=np.array(y_val)
print(X_train_arr.shape)

(51000, 784)


Finally train the data with the new corrected data.

In [62]:
filepath="IIITD_MIDAS_Task2_subpart3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(X_train_arr,y_train_arr,epochs=10,validation_data=(X_val_arr,y_val_arr),verbose=True)

Train on 51000 samples, validate on 9000 samples
Epoch 1/10


22624/51000 [============>.................] - ETA: 7:39 - loss: 0.0342 - acc: 0.968 - ETA: 1:07 - loss: 0.0547 - acc: 0.980 - ETA: 39s - loss: 0.0381 - acc: 0.984 - ETA: 30s - loss: 0.0458 - acc: 0.98 - ETA: 25s - loss: 0.0405 - acc: 0.98 - ETA: 22s - loss: 0.0408 - acc: 0.98 - ETA: 19s - loss: 0.0380 - acc: 0.98 - ETA: 19s - loss: 0.0350 - acc: 0.98 - ETA: 20s - loss: 0.0340 - acc: 0.98 - ETA: 21s - loss: 0.0344 - acc: 0.98 - ETA: 21s - loss: 0.0332 - acc: 0.98 - ETA: 22s - loss: 0.0361 - acc: 0.98 - ETA: 23s - loss: 0.0351 - acc: 0.98 - ETA: 24s - loss: 0.0365 - acc: 0.98 - ETA: 24s - loss: 0.0357 - acc: 0.98 - ETA: 25s - loss: 0.0347 - acc: 0.98 - ETA: 25s - loss: 0.0341 - acc: 0.98 - ETA: 25s - loss: 0.0335 - acc: 0.98 - ETA: 26s - loss: 0.0331 - acc: 0.98 - ETA: 26s - loss: 0.0328 - acc: 0.98 - ETA: 27s - loss: 0.0320 - acc: 0.98 - ETA: 27s - loss: 0.0317 - acc: 0.98 - ETA: 27s - loss: 0.0311 - acc: 0.98 - ETA: 28s - loss: 0.0307 - acc: 0.98 - ETA: 28s - loss: 0.0320 - acc: 0.98 

49856/51000 [============================>.] - ETA: 15s - loss: 0.0366 - acc: 0.98 - ETA: 15s - loss: 0.0367 - acc: 0.98 - ETA: 15s - loss: 0.0367 - acc: 0.98 - ETA: 15s - loss: 0.0366 - acc: 0.98 - ETA: 15s - loss: 0.0365 - acc: 0.98 - ETA: 15s - loss: 0.0364 - acc: 0.98 - ETA: 15s - loss: 0.0364 - acc: 0.98 - ETA: 15s - loss: 0.0362 - acc: 0.98 - ETA: 15s - loss: 0.0362 - acc: 0.98 - ETA: 15s - loss: 0.0361 - acc: 0.98 - ETA: 15s - loss: 0.0360 - acc: 0.98 - ETA: 15s - loss: 0.0358 - acc: 0.98 - ETA: 15s - loss: 0.0357 - acc: 0.98 - ETA: 15s - loss: 0.0357 - acc: 0.98 - ETA: 15s - loss: 0.0358 - acc: 0.98 - ETA: 15s - loss: 0.0357 - acc: 0.98 - ETA: 15s - loss: 0.0356 - acc: 0.98 - ETA: 15s - loss: 0.0355 - acc: 0.98 - ETA: 15s - loss: 0.0357 - acc: 0.98 - ETA: 14s - loss: 0.0358 - acc: 0.98 - ETA: 14s - loss: 0.0359 - acc: 0.98 - ETA: 14s - loss: 0.0362 - acc: 0.98 - ETA: 14s - loss: 0.0362 - acc: 0.98 - ETA: 14s - loss: 0.0362 - acc: 0.98 - ETA: 14s - loss: 0.0364 - acc: 0.98 - ETA

51000/51000 [==============================] - ETA: 0s - loss: 0.0344 - acc: 0.989 - ETA: 0s - loss: 0.0345 - acc: 0.989 - ETA: 0s - loss: 0.0345 - acc: 0.989 - ETA: 0s - loss: 0.0346 - acc: 0.989 - ETA: 0s - loss: 0.0347 - acc: 0.989 - 26s 503us/step - loss: 0.0348 - acc: 0.9892 - val_loss: 0.0501 - val_acc: 0.9877
Epoch 2/10


24192/51000 [=============>................] - ETA: 38s - loss: 0.0172 - acc: 1.00 - ETA: 30s - loss: 0.0045 - acc: 1.00 - ETA: 29s - loss: 0.0220 - acc: 0.99 - ETA: 28s - loss: 0.0172 - acc: 0.99 - ETA: 28s - loss: 0.0318 - acc: 0.98 - ETA: 28s - loss: 0.0285 - acc: 0.98 - ETA: 28s - loss: 0.0263 - acc: 0.98 - ETA: 28s - loss: 0.0236 - acc: 0.99 - ETA: 28s - loss: 0.0225 - acc: 0.99 - ETA: 28s - loss: 0.0331 - acc: 0.98 - ETA: 28s - loss: 0.0304 - acc: 0.98 - ETA: 27s - loss: 0.0279 - acc: 0.99 - ETA: 27s - loss: 0.0257 - acc: 0.99 - ETA: 27s - loss: 0.0255 - acc: 0.99 - ETA: 27s - loss: 0.0245 - acc: 0.99 - ETA: 27s - loss: 0.0241 - acc: 0.99 - ETA: 27s - loss: 0.0229 - acc: 0.99 - ETA: 27s - loss: 0.0218 - acc: 0.99 - ETA: 26s - loss: 0.0235 - acc: 0.99 - ETA: 26s - loss: 0.0227 - acc: 0.99 - ETA: 27s - loss: 0.0223 - acc: 0.99 - ETA: 26s - loss: 0.0272 - acc: 0.99 - ETA: 26s - loss: 0.0263 - acc: 0.99 - ETA: 26s - loss: 0.0259 - acc: 0.99 - ETA: 26s - loss: 0.0250 - acc: 0.99 - ETA

...

48256/51000 [===========================>..] - ETA: 13s - loss: 0.0289 - acc: 0.99 - ETA: 13s - loss: 0.0287 - acc: 0.99 - ETA: 13s - loss: 0.0286 - acc: 0.99 - ETA: 13s - loss: 0.0290 - acc: 0.99 - ETA: 13s - loss: 0.0290 - acc: 0.99 - ETA: 13s - loss: 0.0289 - acc: 0.99 - ETA: 13s - loss: 0.0288 - acc: 0.99 - ETA: 13s - loss: 0.0287 - acc: 0.99 - ETA: 13s - loss: 0.0289 - acc: 0.99 - ETA: 13s - loss: 0.0289 - acc: 0.99 - ETA: 13s - loss: 0.0290 - acc: 0.99 - ETA: 13s - loss: 0.0289 - acc: 0.99 - ETA: 12s - loss: 0.0288 - acc: 0.99 - ETA: 12s - loss: 0.0287 - acc: 0.99 - ETA: 12s - loss: 0.0287 - acc: 0.99 - ETA: 12s - loss: 0.0286 - acc: 0.99 - ETA: 12s - loss: 0.0287 - acc: 0.99 - ETA: 12s - loss: 0.0288 - acc: 0.99 - ETA: 12s - loss: 0.0286 - acc: 0.99 - ETA: 12s - loss: 0.0287 - acc: 0.99 - ETA: 12s - loss: 0.0286 - acc: 0.99 - ETA: 12s - loss: 0.0298 - acc: 0.99 - ETA: 12s - loss: 0.0300 - acc: 0.99 - ETA: 12s - loss: 0.0300 - acc: 0.99 - ETA: 12s - loss: 0.0301 - acc: 0.99 - ETA

51000/51000 [==============================] - ETA: 1s - loss: 0.0296 - acc: 0.991 - ETA: 1s - loss: 0.0296 - acc: 0.991 - ETA: 1s - loss: 0.0298 - acc: 0.991 - ETA: 1s - loss: 0.0298 - acc: 0.991 - ETA: 1s - loss: 0.0297 - acc: 0.991 - ETA: 1s - loss: 0.0299 - acc: 0.991 - ETA: 1s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0301 - acc: 0.991 - ETA: 0s - loss: 0.0300 - acc: 0.991 - ETA: 0s - loss: 0.0300 - acc: 0.991 - ETA: 0s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0298 - acc: 0.991 - ETA: 0s - loss: 0.0299 - acc: 0.991 - ETA: 0s - loss: 0.0298 - acc: 0.991 - ETA: 0s - loss: 0.0298 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA: 0s - loss: 0.0297 - acc: 0.991 - ETA

22368/51000 [============>.................] - ETA: 11s - loss: 5.0589e-04 - acc: 1.00 - ETA: 7s - loss: 0.0193 - acc: 0.9952     - ETA: 11s - loss: 0.0255 - acc: 0.99 - ETA: 14s - loss: 0.0226 - acc: 0.99 - ETA: 16s - loss: 0.0244 - acc: 0.99 - ETA: 17s - loss: 0.0215 - acc: 0.99 - ETA: 19s - loss: 0.0244 - acc: 0.99 - ETA: 19s - loss: 0.0248 - acc: 0.99 - ETA: 20s - loss: 0.0311 - acc: 0.98 - ETA: 21s - loss: 0.0291 - acc: 0.99 - ETA: 21s - loss: 0.0272 - acc: 0.99 - ETA: 21s - loss: 0.0294 - acc: 0.99 - ETA: 22s - loss: 0.0284 - acc: 0.99 - ETA: 22s - loss: 0.0374 - acc: 0.99 - ETA: 22s - loss: 0.0374 - acc: 0.99 - ETA: 22s - loss: 0.0399 - acc: 0.99 - ETA: 23s - loss: 0.0379 - acc: 0.99 - ETA: 23s - loss: 0.0363 - acc: 0.99 - ETA: 23s - loss: 0.0351 - acc: 0.99 - ETA: 23s - loss: 0.0401 - acc: 0.99 - ETA: 23s - loss: 0.0387 - acc: 0.99 - ETA: 23s - loss: 0.0380 - acc: 0.99 - ETA: 23s - loss: 0.0366 - acc: 0.99 - ETA: 24s - loss: 0.0353 - acc: 0.99 - ETA: 24s - loss: 0.0341 - acc: 0

46624/51000 [==========================>...] - ETA: 15s - loss: 0.0266 - acc: 0.99 - ETA: 15s - loss: 0.0266 - acc: 0.99 - ETA: 15s - loss: 0.0266 - acc: 0.99 - ETA: 15s - loss: 0.0265 - acc: 0.99 - ETA: 15s - loss: 0.0264 - acc: 0.99 - ETA: 15s - loss: 0.0264 - acc: 0.99 - ETA: 15s - loss: 0.0263 - acc: 0.99 - ETA: 15s - loss: 0.0262 - acc: 0.99 - ETA: 15s - loss: 0.0262 - acc: 0.99 - ETA: 15s - loss: 0.0262 - acc: 0.99 - ETA: 15s - loss: 0.0261 - acc: 0.99 - ETA: 15s - loss: 0.0262 - acc: 0.99 - ETA: 14s - loss: 0.0262 - acc: 0.99 - ETA: 14s - loss: 0.0262 - acc: 0.99 - ETA: 14s - loss: 0.0265 - acc: 0.99 - ETA: 13s - loss: 0.0265 - acc: 0.99 - ETA: 13s - loss: 0.0264 - acc: 0.99 - ETA: 13s - loss: 0.0264 - acc: 0.99 - ETA: 13s - loss: 0.0264 - acc: 0.99 - ETA: 13s - loss: 0.0265 - acc: 0.99 - ETA: 13s - loss: 0.0271 - acc: 0.99 - ETA: 13s - loss: 0.0270 - acc: 0.99 - ETA: 13s - loss: 0.0270 - acc: 0.99 - ETA: 13s - loss: 0.0271 - acc: 0.99 - ETA: 13s - loss: 0.0272 - acc: 0.99 - ETA

51000/51000 [==============================] - ETA: 2s - loss: 0.0275 - acc: 0.991 - ETA: 2s - loss: 0.0274 - acc: 0.991 - ETA: 2s - loss: 0.0275 - acc: 0.991 - ETA: 2s - loss: 0.0274 - acc: 0.991 - ETA: 2s - loss: 0.0274 - acc: 0.991 - ETA: 2s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0276 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0274 - acc: 0.991 - ETA: 1s - loss: 0.0273 - acc: 0.991 - ETA: 1s - loss: 0.0276 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 1s - loss: 0.0275 - acc: 0.991 - ETA: 0s - loss: 0.0275 - acc: 0.991 - ETA

In [54]:
values=model.evaluate(X_test,y_test)

10000/10000 [==============================] - ETA: 1: - ETA: 2s - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 60us/step


In [59]:
print('Loss:',values[0],'Accuracy:',values[1])

Loss: 0.09303259097627976 Accuracy: 0.9809


Though I had explored some other techniques like the EM technique. I could not apply it completely yet.